In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import statsmodels.api as sm

In [2]:
tweet_data = pd.read_csv('dataWithPolarity.csv')
tweet_data.head()

/Users/anukritigoyal/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,id_x,screen_name,description,created_at_x,location,is_verified,...,created_at_y,tweet_text,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id,sentiment,polarity
0,0,0,0,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2016-07-26 15:15:46,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7.0,4.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
1,1,1,1,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2017-04-06 16:15:01,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5.0,2.0,NaN,NaN,"Sentiment(polarity=-0.1, subjectivity=0.1)",-0.100000
2,2,2,2,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2016-07-27 17:35:17,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1.0,2.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
3,3,3,3,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2015-11-16 17:57:40,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4.0,0.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.3)",0.000000
4,4,4,4,0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,...,2017-01-03 15:59:01,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7.0,3.0,NaN,NaN,"Sentiment(polarity=-0.2916666666666667, subjec...",-0.291667


In [4]:
account_creation = tweet_data['created_at_x'].tolist()
tweet_creation = tweet_data['created_at_y'].tolist()
time = account_creation[0]
# type(time)

In [5]:
# Converting strings of date and times to Datetime objects

accounts = pd.to_datetime(tweet_data['created_at_x'], errors='coerce')
tweet_data['created_at_x'] = accounts
tweets = pd.to_datetime(tweet_data['created_at_y'], errors='coerce')
tweet_data['created_at_y'] = accounts

In [ ]:
# Attaching party of account holders

In [6]:
pol_accounts = pd.read_csv('pol_accounts.csv', sep=';')
pol_tweets = pd.read_csv('pol_tweets.csv', sep=';')

In [7]:
pol_accounts = pol_accounts[pol_accounts['array_agg'].str.contains(r"\b(?:senator|governor|congressperson)\b", regex=True)]

In [8]:
merged = pd.merge(pol_accounts, pol_tweets, left_on = "id", right_on = "user_id", how = "inner")
merged.head()

,id_x,screen_name,description,created_at_x,location,is_verified,latest_following_count,latest_followers_count,latest_status_count,array_agg,id_y,user_id,created_at_y,tweet_text,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",757957591171534848,2962813893,2016-07-26 15:15:46,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7,4,NaN,NaN
1,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",850019020891856902,2962813893,2017-04-06 16:15:01,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5,2,NaN,NaN
2,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",758355089719496705,2962813893,2016-07-27 17:35:17,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1,2,NaN,NaN
3,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",666314203721019393,2962813893,2015-11-16 17:57:40,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4,0,NaN,NaN
4,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",816312921219362816,2962813893,2017-01-03 15:59:01,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7,3,NaN,NaN


In [9]:
# test_suite = merged.iloc[0:9000]
party = []
for index, row in merged.iterrows(): # change from test_suite to merged
    array = row['array_agg']
    if "Republican" in array:
        party.append("Republican")
    elif "Democrat" in array:
        party.append("Democrat")
    else:
        party.append("None")
merged['party'] = party

In [10]:
merged = merged.join(pd.get_dummies(merged.party))

In [10]:
merged['party'].describe()
merged.party.value_counts()

Republican    728863
Democrat      667681
None           33194
Name: party, dtype: int64

In [19]:
# you need to create a regression model for dems and republicans versus polarity
merged.head()

,id_x,screen_name,description,created_at_x,location,is_verified,latest_following_count,latest_followers_count,latest_status_count,array_agg,...,hashtag_entities,url_entities,favorites_count,retweet_count,quoted_status_id,in_reply_to_status_id,party,Democrat,None,Republican
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,{},{https://t.co/vUJ7muCTy5},7,4,NaN,NaN,Republican,0,0,1
1,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,{},{https://medium.com/@RepStefanik/supporting-mi...,5,2,NaN,NaN,Republican,0,0,1
2,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,{NY21},{https://t.co/3CbW2bp1RV},1,2,NaN,NaN,Republican,0,0,1
3,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4,0,NaN,NaN,Republican,0,0,1
4,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,{NY21},{https://t.co/zT5Kj1tmCM},7,3,NaN,NaN,Republican,0,0,1


In [11]:
# tweet_data['id_x'] = tweet_data['id_x'].apply(int)
# merged['id_x'] = merged['id_x'].apply(int)
#tweet_data.head()
merged = merged.iloc[0:1126983]

In [12]:
# inner join merged with dataWithPolarity 

#merged_with_polarity = pd.merge(merged, tweet_data, left_on = "id_x", right_on = "id_x", how = "inner")

merged = merged.join(tweet_data, lsuffix='_x', rsuffix='_y')
merged.head()

,id_x_x,screen_name_x,description_x,created_at_x_x,location_x,is_verified_x,latest_following_count_x,latest_followers_count_x,latest_status_count_x,array_agg_x,...,created_at_y_y,tweet_text_y,hashtag_entities_y,url_entities_y,favorites_count_y,retweet_count_y,quoted_status_id_y,in_reply_to_status_id_y,sentiment,polarity
0,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-01-05 22:17:51,Congrats to Chief Master Sgt Amy Giaquinto of ...,{},{https://t.co/vUJ7muCTy5},7.0,4.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
1,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-01-05 22:17:51,ICYMI I'm working on legislative initiatives t...,{},{https://medium.com/@RepStefanik/supporting-mi...,5.0,2.0,NaN,NaN,"Sentiment(polarity=-0.1, subjectivity=0.1)",-0.100000
2,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-01-05 22:17:51,ICYMI: #NY21 Students register now to particip...,{NY21},{https://t.co/3CbW2bp1RV},1.0,2.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000000
3,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-01-05 22:17:51,The @GOPpolicy hearing on The Essential Role o...,"{Millennials,GOPFuture}",{https://t.co/dWFSDnFJ1e},4.0,0.0,NaN,NaN,"Sentiment(polarity=0.0, subjectivity=0.3)",0.000000
4,2962813893,RepStefanik,Official Twitter Account for Congresswoman Eli...,2015-01-05 22:17:51,NaN,t,246,9499,2082,"{congressperson,Republican,republican}",...,2015-01-05 22:17:51,Today Congress opens the 115th session. Lookin...,{NY21},{https://t.co/zT5Kj1tmCM},7.0,3.0,NaN,NaN,"Sentiment(polarity=-0.2916666666666667, subjec...",-0.291667


In [43]:
toDrop = ['Republican', 'Democrat']

X = test.favorites_count_x 
Y = sm.add_constant(test.polarity)
# Y = sm.add_constant(test.drop(toDrop, axis = 1))
# lm = sm.OLS(X, Y).fit()
# print(lm.summary())

lmMultiple = sm.OLS(X, Y).fit()
print(lmMultiple.summary())


                            OLS Regression Results                            
Dep. Variable:      favorites_count_x   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     3.644
Date:                Mon, 03 Dec 2018   Prob (F-statistic):             0.0563
Time:                        20:57:02   Log-Likelihood:            -9.1573e+06
No. Observations:             1126983   AIC:                         1.831e+07
Df Residuals:                 1126981   BIC:                         1.831e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.3894      0.865     52.452      0.0

In [37]:
print(test.dtypes)

id_x_x                               int64
screen_name_x                       object
description_x                       object
created_at_x_x                      object
location_x                          object
is_verified_x                       object
latest_following_count_x             int64
latest_followers_count_x             int64
latest_status_count_x                int64
array_agg_x                         object
id_y_x                               int64
user_id_x                            int64
created_at_y_x                      object
tweet_text_x                        object
hashtag_entities_x                  object
url_entities_x                      object
favorites_count_x                    int64
retweet_count_x                      int64
quoted_status_id_x                 float64
in_reply_to_status_id_x            float64
party                               object
Democrat                             uint8
None                                 uint8
Republican 

In [44]:
type(test.favorites_count_x[0])

numpy.int64

In [47]:
np.int64(test.Democrat[0])
type(test.Democrat[0])

numpy.uint8

In [20]:
democrat = merged.Democrat
republican = merged.Republican

frame = {"Democrat": democrat, "Republican" : republican}
frame = pd.DataFrame(frame)
frame.head()

,Democrat,Republican
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [26]:
X = merged.polarity
Y = sm.add_constant(frame)
lm = sm.OLS(X, Y).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:               polarity   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     59.76
Date:                Mon, 03 Dec 2018   Prob (F-statistic):           1.11e-26
Time:                        22:14:41   Log-Likelihood:            -2.0977e+05
No. Observations:             1126983   AIC:                         4.195e+05
Df Residuals:                 1126980   BIC:                         4.196e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1299      0.002     70.107      0.0

In [29]:
X_eng = merged.favorites_count_x
Y_eng = sm.add_constant(frame)
lm = sm.OLS(X_eng, Y_eng).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:      favorites_count_x   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     981.9
Date:                Mon, 03 Dec 2018   Prob (F-statistic):               0.00
Time:                        23:25:05   Log-Likelihood:            -9.1563e+06
No. Observations:             1126983   AIC:                         1.831e+07
Df Residuals:                 1126980   BIC:                         1.831e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        176.1899      5.195     33.916      0.0